In [18]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_fcs
from pysd2cat.data import pipeline
from pysd2cat.analysis import biofab_live_dead_analysis as blda

from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob
import pandas as pd
import ast

%load_ext autoreload
%reload_ext autoreload
%autoreload 2



#run_id='r1dd37mcxv5pf4'
run_id='r1dk8xp9dymm54'
work_dir='data/transcriptic/'+run_id
Connection.from_file("~/.transcriptic")
tx_config = json.load(open(os.path.join(expanduser("~"), ".transcriptic")))
transcriptic_email = tx_config['email']
transcriptic_token = tx_config['token']



if not os.path.exists(work_dir):
    os.makedirs(work_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

In [20]:
run_obj = tx_fcs.get_tx_run(run_id)
d = run_obj.data


Attempting to fetch 16 datasets...


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/transcriptic/jupyter/objects.py:433: UserWarning: Missing key 'container' when initializing dataset
  warnings.warn("Missing key {} when initializing dataset".format(e))


In [41]:
def extract_time_point(x):
    return x.Name.split(' ')[3].split('_')[0]
def extract_time(x):
    return x.Name.split(' ')[3].split('_')[1]
def assign_container(prefix, x, containers):
    name = prefix + ' ' + x.time_point
    #print(containers)
    return next(c for c in containers.Containers if name in c.attributes['label'])


try:
    containers = run_obj.containers
except Exception as e:
    containers = run_obj.containers


flow_data = d.loc[d.Operation == 'flow_analyze']
flow_data.loc[:,'time_point'] = flow_data.apply(extract_time_point, axis=1)
flow_data.loc[:,'time'] = flow_data.apply(extract_time, axis=1)
flow_data.loc[:, 'container'] = flow_data.apply(lambda x: assign_container('Flow (Sytox) Plate', x, containers), axis=1)
flow_data.loc[:, 'container_name'] = flow_data.apply(lambda x: x['container'].attributes['label'], axis=1)



flow_data

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Name,DataType,Operation,AnalysisTool,Datasets,time_point,time,container,container_name
4,Flow (Sytox) Plate 1_0.5 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129c126d8>,1,0.5,Container(Flow (Sytox) Plate 1),Flow (Sytox) Plate 1
5,Flow (Sytox) Plate 2_2.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129c12780>,2,2.0,Container(Flow (Sytox) Plate 2),Flow (Sytox) Plate 2
6,Flow (Sytox) Plate 10_50.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129eae8d0>,10,50.0,Container(Flow (Sytox) Plate 10),Flow (Sytox) Plate 10
7,Flow (Sytox) Plate 11_60.5 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129eae748>,11,60.5,Container(Flow (Sytox) Plate 11),Flow (Sytox) Plate 11
8,Flow (Sytox) Plate 12_72.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129be9240>,12,72.0,Container(Flow (Sytox) Plate 12),Flow (Sytox) Plate 12
9,Flow (Sytox) Plate 3_4.5 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129be9f60>,3,4.5,Container(Flow (Sytox) Plate 3),Flow (Sytox) Plate 3
10,Flow (Sytox) Plate 4_8.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x127aa1278>,4,8.0,Container(Flow (Sytox) Plate 4),Flow (Sytox) Plate 4
11,Flow (Sytox) Plate 5_12.5 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129be9860>,5,12.5,Container(Flow (Sytox) Plate 5),Flow (Sytox) Plate 5
12,Flow (Sytox) Plate 6_18.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129c957b8>,6,18.0,Container(Flow (Sytox) Plate 6),Flow (Sytox) Plate 6
13,Flow (Sytox) Plate 7_24.5 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129cc8390>,7,24.5,Container(Flow (Sytox) Plate 7),Flow (Sytox) Plate 7


In [9]:
flow_data.container[4].attributes

{'id': 'ct1dk8xpae4qpph',
 'container_type_id': '96-flat',
 'barcode': None,
 'deleted_at': None,
 'created_at': '2019-09-18T14:48:13.672-07:00',
 'accessions': 0,
 'slot': None,
 'cover': 'low_evaporation',
 'test_mode': False,
 'label': 'Flow (Sytox) Plate 1',
 'location_id': None,
 'shipment_id': None,
 'storage_condition': 'cold_4',
 'shipment_code': None,
 'status': 'pending_destroy',
 'expires_at': None,
 'properties': {},
 'will_be_destroyed_at': None,
 'generated_by_run': {'id': 'r1dk8xp9dymm54',
  'project': {'id': 'p1dk4sbhpavgxn'}},
 'aliquots': [{'id': 'aq1dkcdcyamuf7f',
   'container_id': 'ct1dk8xpae4qpph',
   'well_idx': 0,
   'created_at': '2019-09-19T14:54:40.748-07:00',
   'volume_ul': '100.0',
   'name': None,
   'properties': {'Stain': 'Sytox'},
   'lot_no': None},
  {'id': 'aq1dkcdcyanynbh',
   'container_id': 'ct1dk8xpae4qpph',
   'well_idx': 1,
   'created_at': '2019-09-19T14:54:40.748-07:00',
   'volume_ul': '100.0',
   'name': None,
   'properties': {'Stain': 'S

In [25]:
def get_flow_plate_properties(run_id, transcriptic_email, transcriptic_token, work_dir, download_data=False, container="Flow (Sytox) Plate 1"):
    print("Getting data for container: " + str(container))
    fcs_path = os.path.join(work_dir, container.replace(" ", "_"))
    if not os.path.exists(fcs_path):
        os.makedirs(fcs_path)
    file_info = tx_fcs.create_fcs_manifest_and_get_files(run_id, 
                                                         transcriptic_email, 
                                                         transcriptic_token,
                                                         fcs_path=fcs_path, 
                                                         download_zip=download_data,
                                                         container_name=container
                                                         )
    print(file_info)
    #Flow plate didn't have the properties, so need to get them from another plate
    inoc_plate_info = tx_fcs.get_plate_well_properties(tx_fcs.get_tx_run(run_id), 
                                                       container_name="Inoculation Plate")
    file_info['container'] = container

    for well_id, info in file_info['aliquots'].items():
        info['properties'] = inoc_plate_info[well_id.upper()]
    return file_info

Getting data for container: Flow (Sytox) Plate 1
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 2
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 10
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900

Getting data for container: Flow (Sytox) Plate 11
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_11/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_11/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_11/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_11/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900

Getting data for container: Flow (Sytox) Plate 12
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_12/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_12/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_12/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_12/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900

...

Getting data for container: Flow (Sytox) Plate 5
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 6
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 7
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 8
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 9
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

In [54]:


def make_row(well, info, timeseries_map=False):
    row = { 
        "well" : well,
#        "filename" : info['file'],
#        "checksum" : info['checksum']
        }
    if 'file' in info:
        row['filename'] = info['file']
    if 'checksum' in info:
        row['checksum'] = info['checksum']
    # Properties don't include sytox, so add manually
    def well_stain(well):
        if 'a' in well or 'b' in well or 'c' in well or 'd' in well:
            return "SYTOX Red Stain"
        else:
            return None
        
    row['stain'] = well_stain(well) 
    
    def expand_options(v):
        #print(v)
        kv_pairs = {}
        for option, value in v.items():
            if option == 'Reagents':
                #print(option)
                for reagent, properties in value.items():
                    if reagent == 'Ethanol':
                        kv_pairs['kill'] = reagent
                        kv_pairs['kill_volume'] = properties['final_concentration']['qty']
        return kv_pairs
    
    if timeseries_map:
        row['kill'] = 'Ethanol'
        killed = '9' not in well
        row['kill_volume'] = 300.0 if killed else 0.0 
    else:
        if 'properties' in info:
            for k, v in info['properties'].items():
                if k == 'Options':
                    row.update(expand_options(ast.literal_eval(v)))
                else:
                    row[k] = str(v)
                
                
                
    return row

def make_meta_dataframe(run_id, plate_properties, timeseries_map=False):
    data_df = pd.DataFrame()
    for well, info in plate_properties['aliquots'].items():
        row = make_row(well, info, timeseries_map=timeseries_map)
        data_df = data_df.append(row, ignore_index=True)
        
    data_df['experiment_id'] = run_id
    return data_df
        
def get_container_work_dir(out_dir, container):
    return os.path.join(out_dir, container.replace(" ", "_"))

def get_container_data(out_dir, container):
    path = os.path.join(get_container_work_dir(out_dir, container), 'data.csv')
    df = pd.read_csv(path)
    return df

def make_container_dataframes(run_id, plate_properties_list, out_dir, timeseries_map=False, overwrite=False):
    for plate_properties in plate_properties_list:
        df_path = get_container_work_dir(out_dir, plate_properties['container'])
        if not os.path.exists(df_path):
            os.makedirs(df_path)
        df_file = os.path.join(df_path, 'data.csv')
        if overwrite or not os.path.exists(df_file):
            meta_df = make_meta_dataframe(run_id, plate_properties, timeseries_map=timeseries_map)
            df = pipeline.get_data_and_metadata_df(meta_df, '.', fraction=None, max_records=30000)
            # drop bead controls in column 12
            df = df.loc[~df['well'].str.contains('12')]
            df.to_csv(df_file)

In [55]:
plate_properties = [get_flow_plate_properties(run_id, transcriptic_email, transcriptic_token, work_dir, download_data=False, container=flow_data.container[i].attributes['label']) for i in flow_data.index]
make_container_dataframes(run_id, plate_properties, work_dir, timeseries_map=True, overwrite=True)

Getting data for container: Flow (Sytox) Plate 1
Attempting to fetch 16 datasets...


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/transcriptic/jupyter/objects.py:433: UserWarning: Missing key 'container' when initializing dataset
  warnings.warn("Missing key {} when initializing dataset".format(e))
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/transcriptic/jupyter/objects.py:882: UserWarning: ContainerType given is not supported yet in AP-Py
  warnings.warn("ContainerType given is not supported yet in AP-Py")


{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def881b7c74531e1d5a'}, 'a5': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plat

Getting data for container: Flow (Sytox) Plate 2
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_2/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 10
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_10/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900

...

Getting data for container: Flow (Sytox) Plate 5
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_5/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 6
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_6/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 7
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_7/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 8
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_8/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

Getting data for container: Flow (Sytox) Plate 9
Attempting to fetch 16 datasets...
{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def88

In [56]:
for p in plate_properties:
    df = get_container_data(work_dir, p['container'])
    time_point = flow_data.loc[flow_data['container_name'] == p['container']]['time_point'].iloc[0]
    dead_volumes = list(df.kill_volume.dropna().unique())
    dead_volumes.remove(0.0)
    #print(dead_volumes)
    blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            dead_volumes=dead_volumes,
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                           time_point=time_point)



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 300.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
data_columns: ['Unnamed: 0', 'checksum', 'filename', 'kill', 'kill_volume', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
07:04:51 DEBUG:Splitting Test Harness Data ...
07:04:52 DEBUG:Running Test Harness ...


...


HI
----------------------------------------------------------------------------------------------------
Starting run at time 02:05:41
Normalizing training and testing splits...
Starting Classification training...
Training time was: 158.47 seconds
Testing time was: 5.02 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                       Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  QPk6aRXLZX3z3  2019-09-25  02:05:41  random_forest_classification  Hamed        2           0.751     0.665              0.754      0.436

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
02:08:44 DEBUG:Splitting Tes

['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


02:08:44 DEBUG:Running Test Harness ...



HI
----------------------------------------------------------------------------------------------------
Starting run at time 02:08:44
Normalizing training and testing splits...
Starting Classification training...
Training time was: 164.03 seconds
Testing time was: 5.54 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                         Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  EvgArL187py9x  2019-09-25  02:08:44  random_forest_classification  Hamed        2           0.759     0.631              0.715      0.406              

In [36]:
df.kill_volume.value_counts()

29.0     240000
170.0    240000
105.0    240000
64.0     240000
0.0      240000
Name: kill_volume, dtype: int64

In [44]:
# Multiple time points and 0/15 eth
blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            dead_volumes=[29.0, 64.0, 170.0, 105.0],
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                           time_point=time_point)



{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 29.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 29.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 64.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 64.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 170.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 170.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 105.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'r

In [64]:
# Single time point and Eth titration
blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'])


{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain'}
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': None}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 22:31:18
Normalizing training and testing splits...
Starting Classification training...
Training time was: 40.89 seconds
Testing time was: 0.64 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                       Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  QgDA8YOg58WPP  2019-07-28  22:31:18  random_forest_classification  Hamed        2           0.994     0.994              1.0        1.0                0.994     0.993     

...

HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:35:41
Normalizing training and testing splits...
Starting Classification training...
Training time was: 62.62 seconds
Testing time was: 1.57 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                 Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5ZyJ9Xak7e2xw  2019-07-28  23:35:41  random_forest_classification  Hamed        2           0.647     0.647              0.709      0.677              0.673     

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:36:53
Normalizing training and testing splits...
Starting Classification training...
Training time was: 52.32 seconds
Testing time was: 1.77 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                    Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  52LjE9Q3LRJMM  2019-07-28  23:36:53  random_forest_classification  Hamed        2           0.638     0.638              0.691      0.673              0.64      0.637      0.

In [112]:
df = pd.read_csv(os.path.join(work_dir, 'data.csv'), index_col=0)

pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          output_col = 'live')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


----------------------------------------------------------------------------------------------------
Starting run at time 14:03:08
Normalizing training and testing splits...
Starting Classification training...
Training time was: 41.69 seconds
Testing time was: 0.75 seconds
Prediction time of untested data was: 16.60473394393921

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                      Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5W3jAjw9ez6zr  2019-07-30  14:03:08  random_forest_classification  Hamed        2      

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


----------------------------------------------------------------------------------------------------
Starting run at time 14:04:30
Normalizing training and testing splits...
Starting Classification training...
Training time was: 45.57 seconds
Testing time was: 0.85 seconds
Prediction time of untested data was: 16.545056104660034

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                         Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  EWoyeqNla8WWM  2019-07-30  14:04:30  random_forest_classification  Hamed        2           0.994  

In [116]:
# Train on just side scatter
df = pd.read_csv(os.path.join(work_dir, 'data.csv'), index_col=0)
pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=[ 'SSC-A','SSC-H', 'SSC-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          additional_description={
                                              'channels' : [ 'SSC-A','SSC-H', 'SSC-W']
                                          },
                                          output_col = 'live_ss')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,2,3,4,5,6,7,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True, 'channels': ['SSC-A', 'SSC-H', 'SSC-W']}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index', 'live']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 14:10:11
Normalizing training and testing splits...
Starting Classification training...
Training time was: 27.55 seconds
Testing time was: 0.84 seconds
Prediction time of untested data was: 17.089905261993408

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                               Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  6gD7RVbGQPB6N  2

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 14:11:17
Normalizing training and testing splits...
Starting Classification training...
Training time was: 28.10 seconds
Testing time was: 1.36 seconds
Prediction time of untested data was: 21.104886054992676

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                 Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5RvNyJBOeZwrM  2019-07-30  14:

In [117]:
pred_df.columns

Index(['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index', 'live', 'live_ss'], dtype='object')

In [98]:
len(pred_df.loc[pred_df.stain == 'SYTOX Red Stain'])

1098660